## Criando a camada gold


In [0]:
%sql
use catalog medalhao;
create schema if not exists gold;

## 1º Projeto — Área de Logística (Vendas por
## Localidade)

### 1.1 Criação da tabela gold.ft_vendas_consumidor_local

In [0]:
from pyspark.sql import functions as F

In [0]:
ft_pedidos_total_silver = spark.table("silver.ft_pedido_total")
ft_consumidores_silver = spark.table("silver.ft_consumidores")


df_venda_consumidor_gold = (
    ft_pedidos_total_silver.alias("t").join(
    ft_consumidores_silver.alias("c") , on='id_consumidor', how='left')
     .select(
        F.col("t.id_pedido").cast("string"),
        F.col("t.id_consumidor").cast("string"),
        F.col("t.valor_total_pago_brl").alias("valor_total_pedido_brl").cast("decimal(12,2)"),
        F.col("c.cidade").cast("string"),
        F.col("c.estado").cast("string"),
        F.col("t.data_pedido").cast("date")
    )
)

df_venda_consumidor_gold.write.mode("overwrite").format("delta").saveAsTable("gold.ft_vendas_consumidor_local")


In [0]:
spark.table("gold.ft_vendas_consumidor_local").show(10)


### 1.2 Criação da view
### gold.view_total_compras_por_consumidor



In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_total_compras_por_consumidor AS
SELECT
    cidade,
    estado,
    COUNT(*) AS quantidade_vendas,
    SUM(valor_total_pedido_brl) AS valor_total_localidade
FROM gold.ft_vendas_consumidor_local
GROUP BY cidade, estado;


Criando consulta para vizualizar total de vendas por estado a  partir da view criada

In [0]:
%sql
use catalog medalhao;
SELECT
    estado,
    SUM(quantidade_vendas) AS total_vendas_estado,
    SUM(valor_total_localidade) AS total_valor_vendido_estado
FROM gold.view_total_compras_por_consumidor
GROUP BY estado
ORDER BY total_valor_vendido_estado DESC;

## 2º Projeto — Área de Logística (Análise de Atrasos
## de Entregas)

### 2.1 Criação da tabela
### gold.ft_atrasos_pedidos_local_vendedor:


In [0]:
ft_pedidos_silver = spark.table("silver.ft_pedidos")
ft_consumidores_silver = spark.table("silver.ft_consumidores")
ft_itens_pedidos_silver = spark.table("silver.ft_itens_pedidos")

df_inicial = (
    ft_pedidos_silver.alias("p")
    .join(ft_consumidores_silver.alias("c"), on="id_consumidor", how="left")
    .join(ft_itens_pedidos_silver.alias("i"), on="id_pedido", how="left") 
)

df_atrasos_pedidos_gold = df_inicial.select(
    F.col("p.id_pedido").cast("string"),
    F.col("i.id_vendedor").cast("string"),
    F.col("p.id_consumidor").cast("string"),
    F.col("p.entrega_no_prazo").cast("string"),
    F.col("p.tempo_entrega_dias").cast("integer"),
    F.col("p.tempo_entrega_estimado_dias").cast("integer"),
    F.col("c.cidade").cast("string"),
    F.col("c.estado").cast("string")
)

In [0]:
df_atrasos_pedidos_gold.write.mode("overwrite").format("delta").saveAsTable("gold.ft_atrasos_pedidos_local_vendedor")


In [0]:
# O primeiro foi .show mas nesee ele ficou meio desorganizado no meu entao usei display
display(df_atrasos_pedidos_gold)

### 2.2 Criação das Views Analíticas


2.2.1 gold.view_tempo_medio_entrega_localidade

In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_tempo_medio_entrega_localidade AS
WITH medias AS (
    SELECT
        cidade,
        estado,
        AVG(tempo_entrega_dias) AS tempo_medio_entrega,
        AVG(tempo_entrega_estimado_dias) AS tempo_medio_estimado
    FROM gold.ft_atrasos_pedidos_local_vendedor
    GROUP BY cidade, estado
)

SELECT
    cidade,
    estado,
    tempo_medio_entrega,
    tempo_medio_estimado,
    CASE
        WHEN tempo_medio_entrega IS NULL THEN 'NÃO' -- Considerando os casos em que o pedido nao foi entregue
        WHEN tempo_medio_entrega > tempo_medio_estimado THEN 'SIM'
        ELSE 'NÃO'
    END AS entrega_maior_que_estimado
FROM medias;


Vizualizando view

In [0]:
%sql
SELECT *
FROM gold.view_tempo_medio_entrega_localidade
ORDER BY tempo_medio_entrega DESC;


2.2.2 gold.view_vendedor_pontualidade


In [0]:
%sql
CREATE OR REPLACE VIEW gold.view_vendedor_pontualidade AS
SELECT
    id_vendedor,
    COUNT(*) AS total_pedidos,
    SUM(CASE WHEN entrega_no_prazo IN ('Não', 'Não Entregue') THEN 1 ELSE 0 END) AS total_atrasados,
    ROUND(
        SUM(CASE WHEN entrega_no_prazo IN ('Não', 'Não Entregue') THEN 1 ELSE 0 END) * 100.0 
        / COUNT(*),
        2
    ) AS percentual_atraso
FROM gold.ft_atrasos_pedidos_local_vendedor
GROUP BY id_vendedor; 

Vizualizando view

In [0]:
%sql
SELECT *
FROM gold.view_vendedor_pontualidade
ORDER BY percentual_atraso DESC

## 3º Projeto — Área Comercial (Análises de Vendas
## por Período)

### 3.1 Criação da Dimensão de Tempo — gold.dm_tempo


In [0]:
# Para criar essa dimensao de forma mais eficiente podemos pegar o intervalo de tempo do primeiro e ultimo pedido feito
df_pedidos_silver = spark.table("silver.ft_pedidos")

df_min_max = df_pedidos_silver.select(
    F.min("pedido_compra_timestamp").alias("data_inicio"),
    F.max("pedido_compra_timestamp").alias("data_fim")
)

# Gerando as datas no intervalo de tempo encontrado usando sequence e explode 
df_tempo = (
    df_min_max.select(
        F.explode(
            F.sequence(
                F.to_date("data_inicio"),
                F.to_date("data_fim"),
                F.expr("INTERVAL 1 DAY")
            )
        ).alias("sk_tempo")
    )
)



In [0]:
df_tempo = (
    df_tempo
        .withColumn("ano", F.year("sk_tempo"))
        .withColumn("trimestre", F.quarter("sk_tempo"))
        .withColumn("mes", F.month("sk_tempo"))
        .withColumn("semana_do_ano", F.weekofyear("sk_tempo"))
        .withColumn("dia", F.dayofmonth("sk_tempo"))
        # Dia da semana via EXTRACT
        .withColumn("dow_tmp", F.expr("EXTRACT(DOW FROM sk_tempo)")) 
        .withColumn(
            "dia_da_semana_num",
            F.when(F.col("dow_tmp") == 0, 1)      # domingo
             .otherwise(F.col("dow_tmp") + 1)     # segunda = 2 , sábado = 7
        )
        .drop("dow_tmp")
        
        # Nome do dia da semana (mudar para portugues)
        .withColumn("dia_da_semana_nome", F.date_format("sk_tempo", "EEEE"))
        
        # Nome do mes (mudar para portugues)
        .withColumn("mes_nome", F.date_format("sk_tempo", "MMMM"))
        
        # Indicador de final de semana
        .withColumn(
            "eh_fim_de_semana",
            F.when(F.col("dia_da_semana_num").isin(1,7), "Sim")
             .otherwise("Não")
        )
)


In [0]:
df_tempo.write.mode("overwrite").format("delta").saveAsTable("gold.dm_tempo")


In [0]:
display(df_tempo)
# Colocar nome do dia e mes em portugues